In [2]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [8]:
data = FUSE.dd();
resize!(data.core_profiles.profiles_1d,1)
data.core_profiles.profiles_1d[1].grid.rho_tor_norm = range(0,stop=1,length=10)
data.core_profiles.profiles_1d[1].electrons.temperature = Vector{Float64}(collect(1:10))
data.core_profiles.profiles_1d[1].electrons.temperature(0.5)

data.core_profiles.profiles_1d[1].electrons.temperature = x->x^2
data.core_profiles.profiles_1d[1].electrons.temperature

100-element FUSE.AnalyticalFDVector:
 0.0
 0.0001020304050607081
 0.0004081216202428324
 0.0009182736455463729
 0.0016324864809713297
 0.002550760126517702
 0.0036730945821854917
 0.004999489847974696
 0.006529945923885319
 0.008264462809917356
 0.010203040506070809
 0.012345679012345678
 0.014692378328741967
 ⋮
 0.7901234567901234
 0.8081828384858688
 0.8264462809917354
 0.8449137843077238
 0.8635853484338333
 0.8824609733700643
 0.9015406591164167
 0.9208244056728905
 0.9403122130394859
 0.9600040812162024
 0.9799000102030406
 1.0

In [10]:
Pkg.test("FUSE")

     Testing FUSE
      Status `/private/var/folders/c3/035jbfxd5svg28gdfcc0y1c80000gn/T/jl_3TkYkg/Project.toml`
  [e64856f0] FUSE v0.1.0 `~/.julia/dev/FUSE`
  [a98d9a8b] Interpolations v0.13.4
  [682c06a0] JSON v0.21.2
  [c03570c3] Memoize v0.4.4
  [92933f4c] ProgressMeter v1.7.1
  [67601950] Quadrature v1.9.0
  [8dfed614] Test `@stdlib/Test`
      Status `/private/var/folders/c3/035jbfxd5svg28gdfcc0y1c80000gn/T/jl_3TkYkg/Manifest.toml`
  [621f4979] AbstractFFTs v1.0.1
  [79e6a3ab] Adapt v3.3.1
  [4fba245c] ArrayInterface v3.1.23
  [13072b0f] AxisAlgorithms v1.0.0
  [082447d4] ChainRules v0.8.24
  [d360d2e6] ChainRulesCore v0.10.13
  [861a8166] Combinatorics v1.0.2
  [38540f10] CommonSolve v0.2.0
  [bbf7d656] CommonSubexpressions v0.3.0
  [34da2185] Compat v3.32.0
  [187b0558] ConstructionBase v1.3.0
  [9a962f9c] DataAPI v1.7.0
  [864edb3b] DataStructures v0.18.10
  [e2d170a0] DataValueInterfaces v1.0.0
  [2b5f629d] DiffEqBase v6.70.0
  [163ba53b] DiffResults v1.0.3
  [b552c78f] DiffR

Test Summary: | Pass  Total
FDS           |   14     14
Test Summary: | Pass  Total
FDS_IMAS      |   15     15
JSON_IO: Error During Test at /Users/meneghini/.julia/dev/FUSE/test/runtests_fds.jl:90
  Got exception outside of a @test
  MethodError: Cannot `convert` an object of type Vector{Any} to an object of type FUSE.AbstractFDArray{Float64, 1}
  Closest candidates are:
    convert(::Type{T}, ::LinearAlgebra.Factorization) where T<:AbstractArray at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/factorization.jl:58
    convert(::Type{T}, ::T) where T<:AbstractArray at abstractarray.jl:14
    convert(::Type{T}, ::T) where T at essentials.jl:205
  Stacktrace:
    [1] dict2fuse(dct::Dict{String, Any}, fds::FUSE.equilibrium; verbose::Bool, path::Vector{String})
      @ FUSE ~/.julia/dev/FUSE/src/data.jl:66
    [2] dict2fuse(dct::Dict{String, Any}, fds::FUSE.dd; verbose::Bool, path::Vector{String})
      @ FUSE ~/.julia/dev/FUSE/src/data.j

ERROR: LoadError: LoadError: Some tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.
in expression starting at /Users/meneghini/.julia/dev/FUSE/test/runtests_fds.jl:90
in expression starting at /Users/meneghini/.julia/dev/FUSE/test/runtests.jl:4


LoadError: Package FUSE errored during testing

In [7]:
data = FUSE.dd();
resize!(data.core_profiles.profiles_1d,1)
data.core_profiles.profiles_1d[1].grid.rho_tor_norm = Vector{Float64}(collect(1:10))
data.core_profiles.profiles_1d[1].electrons.temperature = Vector{Float64}(collect(1:10))

┌ Error: Failed to revise /Users/meneghini/.julia/dev/FUSE/src/functionarrays.jl
│   exception = Revise.ReviseEvalException("/Users/meneghini/.julia/dev/FUSE/src/functionarrays.jl:318", ErrorException("invalid redefinition of constant NumericalFDVector"), Any[(top-level scope at functionarrays.jl:318, 1)])
└ @ Revise /Users/meneghini/.julia/packages/Revise/AIcio/src/packagedef.jl:707


LoadError: MethodError: no method matching FUSE.NumericalFDVector(::Vector{Vector{Float64}}, ::Vector{Float64})

In [222]:
Array{Int64,0}

Array{Int64, 0}

In [170]:
filename = joinpath(dirname(dirname(abspath(@__FILE__))), "sample", "sample_equilibrium_ods.json")
@time data = FUSE.json2fuse(filename; verbose=false)

  0.001787 seconds (4.33 k allocations: 320.188 KiB)


EQUILIBRIUM
 time_slice
  [1]
   time ➡ Float64
   profiles_1d
    b_field_max ➡ 17-element Vector{Float64}
    dvolume_drho_tor ➡ 17-element Vector{Float64}
    gm9 ➡ 17-element Vector{Float64}
    dpsi_drho_tor ➡ 17-element Vector{Float64}
    surface ➡ 17-element Vector{Float64}
    rho_tor ➡ 17-element Vector{Float64}
    b_field_min ➡ 17-element Vector{Float64}
    darea_dpsi ➡ 17-element Vector{Float64}
    squareness_upper_inner ➡ 17-element Vector{Float64}
    squareness_lower_inner ➡ 17-element Vector{Float64}
    rho_tor_norm ➡ 17-element Vector{Float64}
    elongation ➡ 17-element Vector{Float64}
    b_field_average ➡ 17-element Vector{Float64}
    psi ➡ 17-element Vector{Float64}
    gm8 ➡ 17-element Vector{Float64}
    dpressure_dpsi ➡ 17-element Vector{Float64}
    triangularity_upper ➡ 17-element Vector{Float64}
    darea_drho_tor ➡ 17-element Vector{Float64}
    area ➡ 17-element Vector{Float64}
    trapped_fraction ➡ 17-element Vector{Float64}
    volume ➡ 17-element V